<a href="https://colab.research.google.com/github/juanxo90/cargamos/blob/main/cargamos_prueba.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prueba técnica para la posición de data science en Cargamos



In [1]:
# importamos las librerias
import pandas as pd
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

import seaborn as sns
sns.set_theme(style="whitegrid")

import ipywidgets as widgets
from ipywidgets import AppLayout, GridspecLayout, Layout, HTML, VBox, Label
# import bqplot as bq

# from branca.element import Template, MacroElement

In [2]:
pd.set_option("display.max_columns", None)

# cargamos los datos
paq_det = pd.read_csv("https://raw.githubusercontent.com/juanxo90/cargamos/main/data/paquete_clean.csv")
reg_det = pd.read_csv("https://raw.githubusercontent.com/juanxo90/cargamos/main/data/register_clean.csv")

In [3]:
paq_det.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14056 entries, 0 to 14055
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   id_paquete        14056 non-null  object
 1   cedis             14056 non-null  object
 2   estado            14056 non-null  object
 3   hora_recolectado  13997 non-null  object
 4   hora_creacion     14056 non-null  object
 5   hora_entrega      12502 non-null  object
 6   hora_falla        1651 non-null   object
dtypes: object(7)
memory usage: 768.8+ KB


In [4]:
reg_det.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15573 entries, 0 to 15572
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   id_paquete          15573 non-null  object
 1   id_monitoreo        15573 non-null  object
 2   arreglo_estado      15573 non-null  object
 3   complemento_estado  15236 non-null  object
 4   fecha_creado        15573 non-null  object
dtypes: object(5)
memory usage: 608.4+ KB


In [5]:
df = reg_det.merge(paq_det, how="inner", on=["id_paquete"])

In [6]:
grouped = df[["arreglo_estado", "complemento_estado", "estado"]].groupby(["arreglo_estado", "complemento_estado", "estado"])["estado"].count()
grouped = pd.DataFrame(grouped)
grouped = grouped.rename(columns={'estado': 'conteo'})
grouped.reset_index(inplace=True)

In [7]:
grouped

,arreglo_estado,complemento_estado,estado,conteo
0,ARRIVAL_SCAN,DELIVERY_ATTEMPTED,FAILED,13
1,ARRIVAL_SCAN,DELIVERY_ATTEMPTED,SUCCESSFUL,2
2,ARRIVAL_SCAN,ORIGIN_SCAN,ERR,21
3,ARRIVAL_SCAN,ORIGIN_SCAN,SCANNED_IN_CEDIS,7
4,ARRIVAL_SCAN,RETURNED_TO_CLIENT,ERR,1
5,ARRIVAL_SCAN,RETURNED_TO_CLIENT,FAILED,1228
6,ARRIVAL_SCAN,RETURNED_TO_CLIENT,ROUTED,1
7,ARRIVAL_SCAN,RETURNED_TO_CLIENT,SCANNED_IN_CEDIS,6
8,ARRIVAL_SCAN,RETURNED_TO_CLIENT,SUCCESSFUL,20
9,ARRIVAL_SCAN,RETURNED_TO_CLIENT,TO_FINAL_CLIENT,3


In [8]:
# Función para desplegar el bar plot
def plot_bar(comp_edo):
    """
    plot barplor
    """
    fig = px.bar(grouped[grouped.complemento_estado == comp_edo], 
                x='arreglo_estado', y='conteo',  color='estado', barmode='group',
                labels={
                        "estado": "Estado",
                        "conteo": "Totales",
                        "arreglo_estado": "Arreglo estado"
                    },
                title='Gráfico del estado en la entrega para {}'.format(comp_edo),
                log_y=True)
    fig.show()

In [18]:
data_canada = px.data.gapminder().query("country == 'Canada'")
fig = px.bar(data_canada, x='year', y='pop')
fig.show()

In [12]:
l = widgets.Text(value="""
Tablero BI sample
""", layout=Layout(width='1000px', height='30px', display="flex", justify_content="center"))
                 
it1 = widgets.Dropdown(options=grouped.complemento_estado.unique(),
                       description='Estado:',disabled=False,
                       layout=Layout(width='300px', height='50px'))
# c1 = widgets.IntSlider(min=10, max = 100, step=5, value = 50, description = 'bins',
#                          orientation='horizontal',layout=Layout(width='300px', height='80px'))
# c1.style.handle_color = 'black'

interactive_bp= widgets.interactive_output(plot_bar, {"comp_edo":it1})
interactive_bp.clear_output(wait = True)
interactive_bp1= widgets.interactive_output(plot_bar, {"comp_edo":it1})


# interactive_hist = widgets.interactive_output(plot_bar, {"comp_edo":it1})
# interactive_hist.clear_output(wait = True)
    
# interactive_box = widgets.interactive_output(plot_bar, {"comp_edo":it1})
# interactive_box.clear_output(wait = True)

# box_layout = Layout(align_items='center',height='110px', width='auto')
# ui1 = widgets.HBox([it1, interactive_box], layout=box_layout) 

grid = GridspecLayout(10,2)

grid[:, 0] = interactive_bp
grid[:, 0].layout.height = 'auto'
grid[0, 1] = it1
# grid[2:, 1] = interactive_bp1
# grid[2:, 1].layout.height = 'auto'
# grid[2:, 1].layout.width = 'auto'

# grid[0, 3:] = interactive_box



display(l,grid)

Text(value='\nTablero BI sample\n', layout=Layout(display='flex', height='30px', justify_content='center', wid…

GridspecLayout(children=(Output(layout=Layout(grid_area='widget001', height='auto')), Dropdown(description='Es…

In [10]:
header = HTML("<h1>Fictional World Weather</h1>", layout=Layout(height='auto'))
header.style.text_align='center'
                 
it1 = widgets.Dropdown(options=grouped.complemento_estado.unique(),
                      disabled=False,
                       layout=Layout(width='180px', height='50px'))
bp = widgets.interactive_output(plot_bar, {"comp_edo":it1})
bp.clear_output(wait = True)
bp1 = widgets.interactive_output(plot_bar, {"comp_edo":it1})
bp1.clear_output(wait = True)

out = HTML(
    value="Hola amigos",
    layout=Layout(width='auto', height='auto')
)


AppLayout(center=bp,
          header=header,
          topleft=VBox([Label("Estado:"),
                             it1]),
          right_sidebar = None
          ,
          footer=out,
          pane_widths=['80px', 1, 1],
          pane_heights=['80px', 4, 1],
          height='580px',
          width='580px',
          grid_gap="20px")

AppLayout(children=(HTML(value='<h1>Fictional World Weather</h1>', layout=Layout(grid_area='header', height='a…